In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install glove_python

     |████████████████████████████████| 266kB 6.1MB/s 
  Created wheel for glove-python: filename=glove_python-0.1.0-cp36-cp36m-linux_x86_64.whl size=700286 sha256=8148384b704c52cd7e4b0a3eb09a765829eb7503b81dcb0f5445b557b58a3105
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [6]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [56]:
#importing various libraries
import nltk
import re
import pandas as pd
import numpy as np
import string
from nltk.tokenize import word_tokenize 
import warnings
from glove import Corpus, Glove

In [57]:
# function for english corpus pre-processing 
def textPreEng(text):
    no_punct = "".join([i for i in text if i not in string.punctuation]) #removing punctuation
    no_numbers="".join([i for i in no_punct if i not in "0123456789"])  #removing numbers
    tokens = re.split('\W+',no_numbers) #generating tokens from sentences
    return(tokens) # return the tokens generated 

In [58]:
# function for hindi corpus pre-processing
def textPreHin(text):
    no_punct = "".join([i for i in text if i not in string.punctuation]) #removing punctuation
    no_numbers="".join([i for i in no_punct if i not in "0123456789"]) #removing numbers
    hindi_actual=re.sub(r'[a-zA-z0-9 ]+'," ",no_numbers) #removing english alphabets if any
    tokens = word_tokenize(hindi_actual) # tokenizing the sentences
    return(tokens) #return the tokens

In [89]:
# training the glove model on english and hindi corpora
def glove(eng_vect,L1_eng,hin_vect,L1_hin):
  eng_cor = Corpus() # object of english corpus
  hin_cor = Corpus() # object of hindi corpus
  
  eng_cor.fit(eng_vect, window=10) #train the english vector to compute the co occurence matrix
  hin_cor.fit(hin_vect, window=10) #train the hindi vector
  
  gloveE = Glove(no_components=4, learning_rate=0.04) #create a Glove object for english to create embeddings
  gloveH = Glove(no_components=4, learning_rate=0.04) #create a Glove object for hindi to create embeddings

  gloveE.fit(eng_cor.matrix, epochs=400, no_threads=5, verbose=False) #fitting the model for english vectors
  gloveH.fit(hin_cor.matrix, epochs=400, no_threads=5, verbose=False) #fitting the model for hindi vectors

  gloveE.add_dictionary(eng_cor.dictionary) #adding the english embeddings generated to the gloveE dictionary
  gloveE.save('gloveE.model') #saving the model

  gloveH.add_dictionary(hin_cor.dictionary) #adding the hindi embeddings generated to the gloveH dictionary
  gloveH.save('gloveH.model')

  # printing the embeddings for english words
  eng_emb = []
  for i in (eng_vect):
    for words in i:
      eng_emb.append(gloveE.word_vectors[gloveE.dictionary[words]])
  print(eng_emb[:50])

  # printing the embeddings for hindi words
  hin_emb = []
  for i in (hin_vect):
    for words in i:
      hin_emb.append(gloveH.word_vectors[gloveH.dictionary[words]])
  print(hin_emb[:50])

  #printing the most similar words of english words from L1
  for i in L1_eng:
    try:
      print(gloveE.most_similar(i))
      count1 = count1+1
    except:
      pass
  #printing the most similar words of hindi words from L1
  for i in L1_hin:
     try:
       print(gloveH.most_similar(i))
       count1 = count1+1
     except:
       pass

In [ ]:
# loading the english corpus to csv file and performing pre-processing on it
eng_corpus = pd.read_csv(r'/content/gdrive/My Drive/NLP_ASSIGNMENT4/english.txt', sep = "\n",encoding="utf8",header = None)
eng_corpus.columns = ['sentences']
eng_corpus['clean text'] = eng_corpus['sentences'].apply(lambda t: textPreEng(t.lower()))
df = eng_corpus['clean text']
eng_vect = df.values.tolist() # converting the tokens into list of lists

# Loading the L1 csv file created in step 1 and converting the words into list
L1 = pd.read_csv(r'/content/gdrive/My Drive/NLP_ASSIGNMENT4/L1.csv')
df1 = L1['english_word']
L1_eng = df1.values.tolist()

L1 = pd.read_csv(r'/content/gdrive/My Drive/NLP_ASSIGNMENT4/L1.csv')
df2 = L1['hindi_word']
L1_hin = df2.values.tolist()

# loading the hindi corpus to csv file and performing pre-processing on it
hin_corpus = pd.read_csv (r'/content/gdrive/My Drive/NLP_ASSIGNMENT4/hindi.txt', sep = "\n",encoding="utf8",header = None)
hin_corpus.columns = ['sentences']
hin_corpus['clean text'] = hin_corpus['sentences'].apply(lambda t: textPreHin(t))
df = hin_corpus['clean text']
hin_vect = df.values.tolist() # converting the tokens into list of lists

# calling the glove function
glove(eng_vect,L1_eng,hin_vect,L1_hin)